In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle


2025-04-15 09:38:16.231920: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 09:38:16.240004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744709896.250176   77069 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744709896.252863   77069 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 09:38:16.263223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:

data = pd.read_csv('Churn_Modelling.csv')
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
onehot_encoder_geo = OneHotEncoder()
geo = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
data = pd.concat([data, geo_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)
X = data.drop(['Exited'], axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geo, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(layers=1,neurons=32,build_fn=create_model, verbose=1)

param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2, 3, 4, 5, 6],
    'epochs': [50, 100],
    'batch_size': [10]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0, cv=3)
grid_result = grid.fit(X_train, y_train)
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
best_model = grid_result.best_estimator_.model
best_model.save('best_model.h5')
best_model.evaluate(X_test, y_test)

/home/shashu/anaconda3/envs/tensorflow/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/shashu/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1744709902.504740   77069 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9706 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/50


I0000 00:00:1744709903.547687   77199 service.cc:148] XLA service 0x7f25a0007ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744709903.547715   77199 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-04-15 09:38:23.558563: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744709903.618244   77199 cuda_dnn.cc:529] Loaded cuDNN version 90300


 96/534 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6555 - loss: 0.6340

I0000 00:00:1744709903.922125   77199 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7286 - loss: 0.5604
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7975 - loss: 0.4412
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8059 - loss: 0.4288
Epoch 4/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8237 - loss: 0.3960
Epoch 5/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8407 - loss: 0.3865
Epoch 6/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8421 - loss: 0.3857
Epoch 7/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8392 - loss: 0.3859
Epoch 8/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8600 - loss: 0.3560
Epoch 9/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8547 - loss: 0.3646
Epoch 10/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8530 - loss: 0.3599
Epoch 11/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8656 - loss: 0.3449
Epoch 12/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

/home/shashu/anaconda3/envs/tensorflow/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/shashu/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5486 - loss: 0.7368
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8057 - loss: 0.4392
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8279 - loss: 0.3933
Epoch 4/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8308 - loss: 0.3929
Epoch 5/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8375 - loss: 0.3771
Epoch 6/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8466 - loss: 0.3679
Epoch 7/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8562 - loss: 0.3506
Epoch 8/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8543 - loss: 0.3557
Epoch 9/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8559 - loss: 0.3509
Epoch 10/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8572 - loss: 0.3448
Epoch 11/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8606 - loss: 0.3416
Epoch 12/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

KeyboardInterrupt: 